In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import torch 
import torch.nn as nn

from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from utils import *
from models import *
from tqdm import tqdm

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
dmin = 2
dmax = 5
m = 8
t = 100
n = 50000
snr = 0
lamda = 0.2
distance = 0.1
coherent = False

lr = 1e-3
wd = 1e-9
nbEpoches = 200
batchSize = 32

array = ULA(m, lamda)
array.build_array(distance)
array.build_array_manifold()

In [4]:
observations, angles, labels = generate_data(n, t, dmin, dmax, snr, snr, array, coherent)

x_train, x_valid, theta_train, theta_valid, label_train, label_valid = train_test_split(observations, angles, labels, test_size=0.2)
x_train, x_test, theta_train, theta_test, label_train, label_test = train_test_split(x_train, theta_train, label_train, test_size=0.2)

train_set = DATASET(x_train, theta_train, label_train)
valid_set = DATASET(x_valid, theta_valid, label_valid)
test_set = DATASET(x_test, theta_test, label_test)

train_loader = DataLoader(train_set, batch_size=batchSize, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batchSize, shuffle=False)

In [5]:
model = DA_MUSIC(dmin, dmax, array).to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

loss_func1 = RMSPE_varied_nbSources(dmin, dmax, dev)
loss_func2 = nn.CrossEntropyLoss()

Loss, Loss_d, Loss_theta, Val_d, Val_theta = [], [], [], [], []

bestVal_d = 0.0
bestVal_theta = 1000.0


for i in tqdm(range(nbEpoches)):

    running_loss = 0.0
    running_loss_d = 0.0
    running_loss_theta = 0.0

    for data in train_loader:
        
        X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
        
        optimizer.zero_grad()
        
        theta_pred, label_pred = model(X)
        
        loss_theta = loss_func1.calculate(theta_pred, theta_true) 
        loss_theta.backward()
        
        loss_d = loss_func2(label_pred, label_true)
        loss_d.backward()
        
        optimizer.step()
        
        running_loss_theta += loss_theta.item()
        running_loss_d += loss_d.item()
        running_loss += loss_theta.item() + loss_d.item()
    
    Loss.append(running_loss/len(train_loader))
    Loss_d.append(running_loss_d/len(train_loader))
    Loss_theta.append(running_loss_theta/len(train_loader))

    with torch.no_grad():

        running_acc_d = 0.0
        running_loss_theta = 0.0

        for data in valid_loader:
            
            X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
            
            theta_pred, label_pred = model(X)
            
            loss_theta = loss_func1.calculate(theta_pred, theta_true) 
            acc_d = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
            
            running_loss_theta += loss_theta.item()
            running_acc_d += acc_d.item()
        
        Val_d.append(running_acc_d/len(valid_loader))
        Val_theta.append(running_loss_theta/len(valid_loader))

        # if Val_d[-1] > bestVal_d:
            
        #     bestVal_d = Val_d[-1]
        #     torch.save(model.state_dict(), "saved_model.pth")

        if Val_theta[-1] < bestVal_theta:
            bestVal_theta = Val_theta[-1]
            torch.save(model.state_dict(), "damusic_{}dB.pth".format(snr))
            count = 0
        else:
            count += 1

        if count == 20:
            model.load_state_dict(torch.load("damusic_{}dB.pth".format(snr), weights_only=True))


        print("RMSPE = {}, Accuracy = {}".format(Val_theta[-1], Val_d[-1]))

  0%|          | 1/200 [00:07<23:57,  7.23s/it]

RMSPE = 0.37443875476194266, Accuracy = 0.3133985623003195


  1%|          | 2/200 [00:14<23:19,  7.07s/it]

RMSPE = 0.2808114337844971, Accuracy = 0.336361821086262


  2%|▏         | 3/200 [00:21<22:58,  7.00s/it]

...

RMSPE = 0.09651826731503581, Accuracy = 0.7564896166134185


100%|█████████▉| 199/200 [21:27<00:06,  6.64s/it]

RMSPE = 0.09444575037914343, Accuracy = 0.7605830670926518


100%|██████████| 200/200 [21:33<00:00,  6.47s/it]

RMSPE = 0.09967936149325234, Accuracy = 0.761082268370607


In [6]:
model = DA_MUSIC(dmin, dmax, array).to(dev)
model.load_state_dict(torch.load("damusic_{}dB.pth".format(snr), weights_only=True))
loss_func_test_global = RMSPE_varied_nbSources(dmin, dmax, dev)

running_acc_d = 0.0
running_loss_theta = 0.0

for data in test_loader:

    X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
    theta_pred, label_pred = model(X)
    loss_theta = loss_func_test_global.calculate(theta_pred, theta_true)
    acc_d = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
    running_loss_theta += loss_theta.item()
    running_acc_d += acc_d.item()

print("Accuracy of DA-MUSIC is {}%".format(running_acc_d/len(test_loader)*100))
print("RMSPE = {}".format(running_loss_theta/len(test_loader)))

Accuracy of DA-MUSIC is 75.3%
RMSPE = 0.09275875145196914


In [7]:
model = DA_MUSIC_v2(dmin, dmax, array).to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

loss_func1 = RMSPE_varied_nbSources(dmin, dmax, dev)
loss_func2 = nn.CrossEntropyLoss()

Loss, Loss_d, Loss_theta, Val_d, Val_theta = [], [], [], [], []

bestVal_d = 0.0
bestVal_theta = 1000.0


for i in tqdm(range(nbEpoches)):

    running_loss = 0.0
    running_loss_d = 0.0
    running_loss_theta = 0.0

    for data in train_loader:
        
        X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
        
        optimizer.zero_grad()
        
        theta_pred, label_pred = model(X)
        
        loss_theta = loss_func1.calculate(theta_pred, theta_true) 
        loss_theta.backward()
        
        loss_d = loss_func2(label_pred, label_true)
        loss_d.backward()
        
        optimizer.step()
        
        running_loss_theta += loss_theta.item()
        running_loss_d += loss_d.item()
        running_loss += loss_theta.item() + loss_d.item()
    
    Loss.append(running_loss/len(train_loader))
    Loss_d.append(running_loss_d/len(train_loader))
    Loss_theta.append(running_loss_theta/len(train_loader))

    with torch.no_grad():

        running_acc_d = 0.0
        running_loss_theta = 0.0

        for data in valid_loader:
            
            X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
            
            theta_pred, label_pred = model(X)
            
            loss_theta = loss_func1.calculate(theta_pred, theta_true) 
            acc_d = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
            
            running_loss_theta += loss_theta.item()
            running_acc_d += acc_d.item()
        
        Val_d.append(running_acc_d/len(valid_loader))
        Val_theta.append(running_loss_theta/len(valid_loader))

        # if Val_d[-1] > bestVal_d:
            
        #     bestVal_d = Val_d[-1]
        #     torch.save(model.state_dict(), "saved_model.pth")

        if Val_theta[-1] < bestVal_theta:
            bestVal_theta = Val_theta[-1]
            torch.save(model.state_dict(), "damusic_v2_{}dB.pth".format(snr))
            count = 0
        else:
            count += 1

        if count == 20:
            model.load_state_dict(torch.load("damusic_v2_{}dB.pth".format(snr), weights_only=True))


        print("RMSPE = {}, Accuracy = {}".format(Val_theta[-1], Val_d[-1]))

  0%|          | 0/200 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 must have the same dtype, but got ComplexFloat and Float

In [ ]:
model = DA_MUSIC_v2(dmin, dmax, array).to(dev)
model.load_state_dict(torch.load("damusic_v2_{}dB.pth".format(snr), weights_only=True))
loss_func_test_global = RMSPE_varied_nbSources(dmin, dmax, dev)

running_acc_d = 0.0
running_loss_theta = 0.0

for data in test_loader:

    X, theta_true, label_true = data[0].to(dev), data[1].to(dev), data[2].to(dev)
    theta_pred, label_pred = model(X)
    loss_theta = loss_func_test_global.calculate(theta_pred, theta_true)
    acc_d = (torch.argmax(label_pred, dim=1) == torch.argmax(label_true, dim=1)).float().mean()
    running_loss_theta += loss_theta.item()
    running_acc_d += acc_d.item()

print("Accuracy of DA-MUSIC is {}%".format(running_acc_d/len(test_loader)*100))
print("RMSPE = {}".format(running_loss_theta/len(test_loader)))

In [ ]:
nbSources_test = torch.argmax(theta_test, dim=1)

aic_test = AIC(x_test)
aic_acc = (aic_test == nbSources_test).float().mean()
print("Accuracy of AIC estimation is {}%".format(aic_acc * 100))

In [ ]:
mdl_test = MDL(x_test)
mdl_acc = (mdl_test == nbSources_test).float().mean()
print("Accuracy of MDL estimation is {}%".format(mdl_acc * 100))

In [ ]:
loss_func_test = RMSPE_varied_nbSources_test(dmin, dmax, dev)

for data in test_loader:

    X, theta_true = data[0].to(dev), data[1].to(dev)
    theta_pred, label_pred = model(X)
    loss_func_test.calculate(theta_pred, label_pred, theta_true)

loss_func_test.resume()